Initial setup

In [33]:
import pandas as pd

In [34]:
import numpy as np

In [35]:
u_cols=['user_id','age','sex','occupation','zip_code']
users=pd.read_csv(r"C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Recommendation engine\ml-100k\ml-100k\u.user",sep='|',names=u_cols, encoding='latin-1')

In [36]:
#read rating files
r_cols=['user_id','movie_id','rating','unix_timestamp']
ratings=pd.read_csv(r"C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Recommendation engine\ml-100k\ml-100k\u.data", sep='\t', names=r_cols, encoding='latin-1')


In [37]:
#reading item files
i_cols=['movie_id','movie title','release date','video release date','IMDB url','unknown','Action','Adventure','Animation','Children','Comdey',
        'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
items=pd.read_csv(r"C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Recommendation engine\ml-100k\ml-100k\u.item", sep='|', names=i_cols, encoding='latin-1')

In [38]:
print("\nUser Data :")
print("shape:", ratings.shape)
print(ratings.head())


User Data :
shape: (100000, 4)
   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596


In [39]:
print("\nItem Data :")
print("shape ",items.shape)
print(items.head())


Item Data :
shape  (1682, 24)
   movie_id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDB url  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Children  ...  Fantasy  Film-Noir  Horror  Musical  \
0          0          1         1  ...        0 

In [40]:
r_cols=['user_id','movie_id','rating','unix_timestamp']
ratings_train=pd.read_csv(r"C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Recommendation engine\ml-100k\ml-100k\ua.base", sep='\t', names=r_cols, encoding='latin-1')
ratings_test=pd.read_csv(r"C:\Users\prasa\OneDrive\Desktop\DSA hackathons\Recommendation engine\ml-100k\ml-100k\ua.test", sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape
ratings_test.shape

(9430, 4)

Collaborative siltering model

In [41]:
n_users=ratings.user_id.unique().shape[0]
n_items=ratings.movie_id.unique().shape[0]

In [42]:
data_matrix=np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1]=line[3]

In [43]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')


In [44]:
def predict(ratings, similarity, type='user'):
    if type=='user':
        mean_user_rating=ratings.mean(axis=1)
        ratings_diff=(ratings-mean_user_rating[:, np.newaxis])
        pred=mean_user_rating[:,np.newaxis]+similarity.dot(ratings_diff)
    elif type=='item':
        pred=ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    return pred
    

In [45]:
user_prediction=predict(data_matrix, user_similarity, type='user')
item_prediction=predict(data_matrix, item_similarity, type='item')

Building a model

In [46]:
class MF():

    # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
        self.sgd()
        mse = self.mse()
        training_process.append((i, mse))
        if (i+1) % 20 == 0:
            print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Ratings for user i and moive j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [47]:
R=np.array(ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0))

In [48]:
mf=MF(R, K=20, alpha=0.001, beta=0.01, iterations=100)
training_process=mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 100 ; error = 337.5685

P x Q:
[[3.66795927 3.50933911 3.51102061 ... 3.54774432 3.5366693  3.5371693 ]
 [3.66765722 3.51586989 3.49672415 ... 3.52041164 3.53708066 3.54793684]
 [3.60684295 3.45873734 3.44013847 ... 3.46958453 3.48271411 3.4976175 ]
 ...
 [3.6813511  3.51282801 3.4885697  ... 3.5145044  3.53965348 3.53249836]
 [3.71908948 3.54926761 3.55043679 ... 3.59062415 3.59131418 3.58097309]
 [3.64498752 3.48386914 3.46356526 ... 3.52328473 3.48739103 3.49847721]]



using the package

Popularity recommender

In [49]:
train_data = pd.DataFrame(ratings_train)
test_data = pd.DataFrame(ratings_test)

In [55]:
train_data

,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
90565,943,1047,2,875502146
90566,943,1074,4,888640250
90567,943,1188,3,888640250
90568,943,1228,3,888640275


In [50]:
popularity_model = train_data.groupby('movie_id')['rating'].mean().reset_index()

# Rename the column to 'mean_rating'
popularity_model.rename(columns={'rating': 'mean_rating'}, inplace=True)

# Sort the DataFrame by 'mean_rating' in descending order
popularity_model.sort_values('mean_rating', ascending=False, inplace=True)

In [51]:
# Get the top 5 movies
top_5_movies = popularity_model.head(5)

# Print the top 5 movies for each user
for user in [1, 2, 3, 4, 5]:
    print(f"Top 5 movie recommendations for user {user}:")
    print(top_5_movies)
    print("\n")


Top 5 movie recommendations for user 1:
      movie_id  mean_rating
1499      1500          5.0
1188      1189          5.0
1200      1201          5.0
1121      1122          5.0
1466      1467          5.0


Top 5 movie recommendations for user 2:
      movie_id  mean_rating
1499      1500          5.0
1188      1189          5.0
1200      1201          5.0
1121      1122          5.0
1466      1467          5.0


Top 5 movie recommendations for user 3:
      movie_id  mean_rating
1499      1500          5.0
1188      1189          5.0
1200      1201          5.0
1121      1122          5.0
1466      1467          5.0


Top 5 movie recommendations for user 4:
      movie_id  mean_rating
1499      1500          5.0
1188      1189          5.0
1200      1201          5.0
1121      1122          5.0
1466      1467          5.0


Top 5 movie recommendations for user 5:
      movie_id  mean_rating
1499      1500          5.0
1188      1189          5.0
1200      1201          5.0
1121    

In [52]:
from sklearn.metrics.pairwise import cosine_similarity


# Assuming that 'train_data' is a matrix of user-item interactions
# Calculate the cosine similarity between items
item_similarity = cosine_similarity(train_data.T)
item_similarity_df = pd.DataFrame(item_similarity, index=train_data.columns, columns=train_data.columns)

# Function to return the top-N items for a user
def recommend_items(user_index, n=5):
    # Get the items already rated by this user
    known_items = train_data.loc[user_index]
    known_items = known_items[known_items > 0].index.values

    # Get the predicted ratings for all other items
    predicted_ratings = item_similarity_df.loc[known_items].sum()
    predicted_ratings = predicted_ratings.drop(known_items)

    # Get the top-N items
    recommended_items = predicted_ratings.nlargest(n)
    return recommended_items

# Get the top-5 items for users 1, 2, 3, 4, 5
for user in [1, 2, 3, 4, 5]:
    print(f"Top 5 item recommendations for user {user}:")
    print(recommend_items(user, 5))
    print("\n")


Top 5 item recommendations for user 1:
Series([], dtype: float64)


Top 5 item recommendations for user 2:
Series([], dtype: float64)


Top 5 item recommendations for user 3:
Series([], dtype: float64)


Top 5 item recommendations for user 4:
Series([], dtype: float64)


Top 5 item recommendations for user 5:
Series([], dtype: float64)




In [53]:
def recommend_items(user_index, n=5):
    # Get the items already rated by this user
    known_items = train_data.loc[user_index]
    known_items = known_items[known_items > 0].index.values

    if len(known_items) == 0:
        return pd.Series([])  # Return an empty series if the user has not rated any items

    # Get the predicted ratings for all other items
    predicted_ratings = item_similarity_df.loc[known_items].sum()
    predicted_ratings = predicted_ratings.drop(known_items)

    if predicted_ratings.empty:
        return pd.Series([])  # Return an empty series if there are no ratings to predict

    # Get the top-N items
    recommended_items = predicted_ratings.nlargest(n)
    return recommended_items


In [54]:
for user in [1, 2, 3, 4, 5]:
    print(f"Top 5 item recommendations for user {user}:")
    print(recommend_items(user, 5))
    print("\n")

Top 5 item recommendations for user 1:
Series([], dtype: object)


Top 5 item recommendations for user 2:
Series([], dtype: object)


Top 5 item recommendations for user 3:
Series([], dtype: object)


Top 5 item recommendations for user 4:
Series([], dtype: object)


Top 5 item recommendations for user 5:
Series([], dtype: object)




In [56]:
def recommend_items(user_index, n=5):
    # Get the items already rated by this user
    known_items = train_data.loc[user_index]
    known_items = known_items[known_items > 0].index.values

    if len(known_items) == 0:
        return pd.DataFrame(columns=['user_id', 'movie_id', 'score', 'rank'])  # Return an empty DataFrame if the user has not rated any items

    # Get the predicted ratings for all other items
    predicted_ratings = item_similarity_df.loc[known_items].sum()
    predicted_ratings = predicted_ratings.drop(known_items)

    if predicted_ratings.empty:
        return pd.DataFrame(columns=['user_id', 'movie_id', 'score', 'rank'])  # Return an empty DataFrame if there are no ratings to predict

    # Get the top-N items
    recommended_items = predicted_ratings.nlargest(n)

    # Create a DataFrame with user_id, movie_id, score, and rank
    recommendations = pd.DataFrame({
        'user_id': user_index,
        'movie_id': recommended_items.index,
        'score': recommended_items.values,
        'rank': range(1, len(recommended_items) + 1)
    })

    return recommendations

# Get the top-5 items for users 1, 2, 3, 4, 5
for user in [1, 2, 3, 4, 5]:
    print(f"Top 5 item recommendations for user {user}:")
    print(recommend_items(user, 5))
    print("\n")


Top 5 item recommendations for user 1:
Empty DataFrame
Columns: [user_id, movie_id, score, rank]
Index: []


Top 5 item recommendations for user 2:
Empty DataFrame
Columns: [user_id, movie_id, score, rank]
Index: []


Top 5 item recommendations for user 3:
Empty DataFrame
Columns: [user_id, movie_id, score, rank]
Index: []


Top 5 item recommendations for user 4:
Empty DataFrame
Columns: [user_id, movie_id, score, rank]
Index: []


Top 5 item recommendations for user 5:
Empty DataFrame
Columns: [user_id, movie_id, score, rank]
Index: []


